# 1) Import libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# 2) Load the CSV

In [ ]:
path = "/content/Smart_Home_Dataset.csv"
df = pd.read_csv(path, low_memory=False)

In [ ]:
df

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503906,1452128306,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,...,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503907,1452128307,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,...,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503908,1452128308,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,...,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503909,1452128309,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,...,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51


# Clean column names from the spaces

In [ ]:
df.columns = df.columns.str.strip()

# Display the Dataframe details

In [ ]:
print("Loaded shape:", df.shape)
print("First columns:", df.columns[:10])

Loaded shape: (503911, 32)
First columns: Index(['time', 'use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]'],
      dtype='object')


# Fix the time column (Unix timestamp -> datetime)

In [ ]:
# 'time column' in this dataset is Unix epoch (seconds), e.g., 1452115920
df["time"] = pd.to_numeric(df["time"], errors="coerce")
df = df.dropna(subset=["time"]).copy()
df['time']


,time
0,1.451624e+09
1,1.451624e+09
2,1.451624e+09
3,1.451624e+09
4,1.451624e+09
...,...
503905,1.452128e+09
503906,1.452128e+09
503907,1.452128e+09
503908,1.452128e+09


# Convert to datetime (unit='s' because values are 10-digit seconds)

In [ ]:
df["time"] = pd.to_datetime(df["time"].astype("int64"), unit="s")

In [ ]:
df["time"]

,time
0,2016-01-01 05:00:00
1,2016-01-01 05:00:01
2,2016-01-01 05:00:02
3,2016-01-01 05:00:03
4,2016-01-01 05:00:04
...,...
503905,2016-01-07 00:58:25
503906,2016-01-07 00:58:26
503907,2016-01-07 00:58:27
503908,2016-01-07 00:58:28


# Create time-based features

In [ ]:
df["hour"] = df["time"].dt.hour
df["dayofweek"] = df["time"].dt.dayofweek

In [ ]:
print("Time range:", df["time"].min(), "->", df["time"].max())

Time range: 2016-01-01 05:00:00 -> 2016-01-07 00:58:29


# Identify device columns (kW columns) and exclude totals

In [ ]:
# Find all columns ending with [kW]
kw_cols = [c for c in df.columns if c.endswith("[kW]")]

In [ ]:
# Exclude general/aggregate columns (these are NOT single devices)
exclude_kw = {"use [kW]", "gen [kW]", "House overall [kW]", "Solar [kW]"}
device_cols = [c for c in kw_cols if c not in exclude_kw]

In [ ]:
print("Number of device columns:", len(device_cols))
print("Example device columns:", device_cols[:10])


Number of device columns: 14
Example device columns: ['Dishwasher [kW]', 'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]', 'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]', 'Kitchen 14 [kW]', 'Kitchen 38 [kW]']


# 4) Pick weather columns that exist in the dataset

In [ ]:
candidate_weather_cols = [
    "temperature", "humidity", "visibility", "pressure", "windSpeed",
    "cloudCover", "windBearing", "precipIntensity", "dewPoint",
    "precipProbability", "apparentTemperature"
]
weather_cols = [c for c in candidate_weather_cols if c in df.columns]

print("Weather columns used:", weather_cols)

Weather columns used: ['temperature', 'humidity', 'visibility', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability', 'apparentTemperature']


# 5) Convert wide -> long (one row = one device reading)

In [ ]:
# id columns are metadata we keep for every row
id_cols = ["time", "hour", "dayofweek"] + weather_cols

df_long = df.melt(
    id_vars=id_cols,
    value_vars=device_cols,
    var_name="device",
    value_name="energy_kw"
)

In [ ]:
# Clean device names (remove " [kW]" suffix)
df_long["device"] = df_long["device"].str.replace(" [kW]", "", regex=False)

In [ ]:
# Ensure energy is numeric
df_long["energy_kw"] = pd.to_numeric(df_long["energy_kw"], errors="coerce")

In [ ]:
# Drop rows where energy is missing (optional, but recommended)
df_long = df_long.dropna(subset=["energy_kw"]).copy()

In [ ]:
print("Long shape:", df_long.shape)
print(df_long[["time", "device", "energy_kw"]].head())

Long shape: (7054740, 16)
                 time      device  energy_kw
0 2016-01-01 05:00:00  Dishwasher   0.000033
1 2016-01-01 05:00:01  Dishwasher   0.000000
2 2016-01-01 05:00:02  Dishwasher   0.000017
3 2016-01-01 05:00:03  Dishwasher   0.000017
4 2016-01-01 05:00:04  Dishwasher   0.000133


# 6) Create temperature bins (temp_bin)

In [ ]:
# We bin temperature so we can group "similar weather" together.
# This avoids overly sparse combinations.
if "temperature" in df_long.columns:
    df_long["temp_bin"] = pd.cut(
        df_long["temperature"],
        bins=[-50, 0, 10, 20, 30, 50],
        labels=["<0", "0-10", "10-20", "20-30", "30+"]
    )
else:
    # If temperature doesn't exist for some reason, create a dummy bin
    df_long["temp_bin"] = "NA"


# 7) Apply PIVOT: Build baseline using pivot_table (device+hour+temp_bin)

In [ ]:
# Pivot table gives "normal consumption baseline" per context.
baseline = df_long.pivot_table(
    index=["device", "hour", "temp_bin"],
    values="energy_kw",
    aggfunc=["mean", "std"],
    observed=True
).reset_index()

baseline.columns = ["device", "hour", "temp_bin", "baseline_mean", "baseline_std"]

# Merge baseline back to each row (so every reading has a baseline)

In [ ]:
df_feat = df_long.merge(
    baseline,
    on=["device", "hour", "temp_bin"],
    how="left"
)

In [ ]:
# Safe std (avoid 0 or NaN)
df_feat["baseline_std_safe"] = df_feat["baseline_std"].fillna(0)
df_feat.loc[df_feat["baseline_std_safe"] == 0, "baseline_std_safe"] = 1e-6

In [ ]:
# Compute z-score relative to baseline
df_feat["z_pivot"] = (df_feat["energy_kw"] - df_feat["baseline_mean"]) / df_feat["baseline_std_safe"]


In [ ]:
# Label anomalies with pivot baseline
df_feat["anomaly_pivot"] = (df_feat["z_pivot"].abs() > 2).astype(int)

print("Anomaly rate (pivot):", df_feat["anomaly_pivot"].mean())

Anomaly rate (pivot): 0.020698991032979244


# 8) Define features for models

In [ ]:
# base features
base_features = ["hour", "dayofweek"]

In [ ]:
# Add weather features if they exist
for c in ["temperature", "humidity"]:
    if c in df_long.columns:
        base_features.append(c)

In [ ]:
# base features + baseline features
pivot_features = base_features + ["baseline_mean", "baseline_std"]
X_pivot = df_feat[pivot_features]
y_pivot = df_feat["anomaly_pivot"]

# 9) Train/Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pivot, y_pivot, test_size=0.2, random_state=42, stratify=y_pivot)

# 10) Train ML model (Random Forest Classifier)

In [ ]:
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    class_weight="balanced"
)


model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       random_state=42)

# 11) Evaluate and compare results

In [ ]:
pred = model.predict(X_test)


print(" MODEL : PIVOT / BASELINE")
print("====================")
print(classification_report(y_test, pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))


 MODEL : PIVOT / BASELINE
              precision    recall  f1-score   support

           0       1.00      0.77      0.87   1381743
           1       0.08      0.90      0.14     29205

    accuracy                           0.77   1410948
   macro avg       0.54      0.84      0.50   1410948
weighted avg       0.98      0.77      0.85   1410948

Confusion Matrix:
 [[1060188  321555]
 [   2834   26371]]
